In [4]:
import os
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from tqdm import tqdm  
from torchvision.models import alexnet, AlexNet_Weights
from torch.optim.lr_scheduler import ReduceLROnPlateau
from collections import Counter
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader

Dataset Processing and Parameters

In [5]:
data_dir = "../mri-images/augmented-images-v3"
batch_size = 32
num_epochs = 30
learning_rate = 0.001  # Learning rate parameter


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
# Transformations: replicate grayscale channels to match AlexNet's input requirements (3 channels)
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Replicate grayscale to 3 channels
    transforms.Resize((224, 224)),               # Resize to AlexNet input size
    transforms.ToTensor(),                       # Convert to tensor
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Normalize channels
])

# Load dataset
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Prepare 5-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Results dictionary to store validation results for each fold
results = {}

# Perform cross-validation and store metrics for all folds
all_train_losses, all_val_losses = [], []
all_train_accuracies, all_val_accuracies = [], []

In [8]:
print(f"Number of classes: {len(dataset.classes)}")
print(f"Class names: {dataset.classes}")

Number of classes: 3
Class names: ['AD', 'CN', 'MCI']


In [9]:
class_counts = Counter(dataset.targets)
print(f"Number of images in each class: {class_counts}")

Number of images in each class: Counter({2: 932, 0: 916, 1: 916})


In [ ]:
screen_width = 80
for fold_idx, (train_idx, val_idx) in enumerate(kf.split(dataset)):
    # Create the fold header with "--" padding
    fold_header = f" Fold {fold_idx+1}/{kf.n_splits} "
    padding = (screen_width - len(fold_header)) // 2
    print(f"{'-' * padding}{fold_header}{'-' * padding}")
    
    # Split dataset into train and validation sets based on indices
    train_subset = torch.utils.data.Subset(dataset, train_idx)
    val_subset = torch.utils.data.Subset(dataset, val_idx)
    
    train_loader = torch.utils.data.DataLoader(train_subset, batch_size=batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_subset, batch_size=batch_size, shuffle=False)
    
    # Load AlexNet with updated pre-trained weights
    # Instantiate a new AlexNet model for each fold
    weights = AlexNet_Weights.DEFAULT 
    model = alexnet(weights=weights)
    model.classifier[6] = nn.Linear(4096, len(dataset.classes))  # Modify final layer to match number of classes
    model = model.to(device)
    
    criterion = nn.CrossEntropyLoss()
    
    # Use SGD optimizer with momentum
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=1e-4)

    # Initialize the StepLR scheduler
    # scheduler = StepLR(optimizer, step_size=5, gamma=0.5)  # Reduce LR by 0.5 every 5 epochs

    # Add a ReduceLROnPlateau scheduler
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)
    
    # Store metrics for plotting later
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []
    
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        
        # Training phase with tqdm progress bar
        model.train()
        running_loss, correct_preds, total_samples = 0.0, 0, 0
        
        train_loader_tqdm = tqdm(train_loader, desc="Training", leave=False)
        for inputs, labels in train_loader_tqdm:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct_preds += (preds == labels).sum().item()
            total_samples += labels.size(0)
        
        epoch_train_loss = running_loss / total_samples
        epoch_train_acc = correct_preds / total_samples
        
        train_losses.append(epoch_train_loss)
        train_accuracies.append(epoch_train_acc)

        print(f"Train Loss: {epoch_train_loss:.5f}, Train Accuracy: {epoch_train_acc:.5f}")
      
        # Validation phase with tqdm progress bar
        model.eval()
        running_loss, correct_preds, total_samples = 0.0, 0, 0
        
        val_loader_tqdm = tqdm(val_loader, desc="Validation", leave=False)
        with torch.no_grad():
            for inputs, labels in val_loader_tqdm:
                inputs, labels = inputs.to(device), labels.to(device)
                
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            
                running_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                correct_preds += (preds == labels).sum().item()
                total_samples += labels.size(0)
        
        epoch_val_loss = running_loss / total_samples
        epoch_val_acc = correct_preds / total_samples
        
        val_losses.append(epoch_val_loss)
        val_accuracies.append(epoch_val_acc)

        print(f"Val Loss: {epoch_val_loss:.4f}, Val Accuracy: {epoch_val_acc:.4f}")
        scheduler.step(epoch_val_loss)

        # Log the updated learning rate
        current_lr = scheduler.get_last_lr()
        print(f"Epoch {epoch+1}: Current Learning Rate: {current_lr}")

    all_train_losses.append(train_losses)
    all_val_losses.append(val_losses)
    all_train_accuracies.append(train_accuracies)
    all_val_accuracies.append(val_accuracies)

    # Store validation accuracy of this fold in the results dictionary
    results[f"Fold {fold_idx+1}"] = epoch_val_acc

Fold 1/5
Epoch 1/30


Train Loss: 1.13843, Train Accuracy: 0.34916


Val Loss: 1.09918, Val Accuracy: 0.34901
Epoch 1: Current Learning Rate: [0.001]
Epoch 2/30


Train Loss: 1.09105, Train Accuracy: 0.37992


Val Loss: 1.08869, Val Accuracy: 0.37432
Epoch 2: Current Learning Rate: [0.001]
Epoch 3/30


...

Val Loss: 1.04623, Val Accuracy: 0.61775
Epoch 28: Current Learning Rate: [0.00025]
Epoch 29/30


Train Loss: 0.15924, Train Accuracy: 0.94756


Val Loss: 1.12114, Val Accuracy: 0.60688
Epoch 29: Current Learning Rate: [0.000125]
Epoch 30/30


Train Loss: 0.14119, Train Accuracy: 0.95705


Val Loss: 1.00727, Val Accuracy: 0.63949
Epoch 30: Current Learning Rate: [0.000125]


In [ ]:
# Calculate average accuracy across all folds
average_accuracy = np.mean(list(results.values()))

# Print results for each fold and average accuracy
print("\nValidation Results:")
for fold_name, accuracy in results.items():
    print(f"{fold_name}: {accuracy:.4f}")
print(f"Average Validation Accuracy: {average_accuracy:.4f}")


Validation Results:
Fold 1: 0.5967
Fold 2: 0.6166
Fold 3: 0.5353
Fold 4: 0.5913
Fold 5: 0.6395
Average Accuracy: 0.5959
